In [1]:
TestFile='./Training/W2VAsFeatures-ChallengeSetFeatureFile.txt'
with open(TestFile) as f:
    test = f.readlines()

In [12]:
ScoresFile='./Training/W2VAsFeaturesChallenge-2080Model-2080Model-100Trees-NDCG500-tc-1-lr01-leaf50.txt'
with open(ScoresFile) as f:
    scores = f.readlines()

In [13]:
PidTestTracks={}
for l in test:
    pid=l.split()[1].split(':')[1].strip()
    track=l.split('#')[1].strip()
    PidTestTracks.setdefault(pid,[]).append(track)

In [14]:
PidTracksScores={}
for l in scores:
    pid=l.split()[0].strip()
    trackScore=l.split()[2].strip()
    PidTracksScores.setdefault(pid,[]).append(float(trackScore))

In [15]:
from random import shuffle
rerankedCandidates={}
for pid,tracksList  in PidTestTracks.items():
    scoresList=PidTracksScores[pid]
    zippedPairs=zip(tracksList,scoresList)
    shuffle(zippedPairs)
    rerankedCandidates[pid]=[x[0] for x in sorted(zippedPairs, key=lambda x: x[1], reverse=True)]

In [16]:
len(tracksList), len(set(tracksList))

(1698, 1698)

In [17]:
count=0
for pid in rerankedCandidates:
    if len(set(rerankedCandidates[pid]))<500:
        count+=1
print count

8


In [18]:
#load test dataset
import itertools as it
import sys
import json
import re
import collections
import os
from collections import Counter

filename = '/home/ubuntu/testData/challenge_set.json'

f = open(filename)
js = f.read()
f.close()
mpd_slice = json.loads(js)

count=0
testPidMeta={}

for playlist in mpd_slice['playlists']:
    playlistId=str(playlist['pid'])
    trackSet=[]
    
    for track in playlist['tracks']:
        trackSet.append(track['track_uri'].replace('spotify:track:',''))
        
    if len(trackSet)==0:
        testType='Challenge'
        qtype='TitleOnly'
        title=True
        listLen=0
    if 'name' in playlist and len(trackSet)>0:
        testType='Challenge'
        qtype='TitleAndTracks'
        title=True
        listLen=len(trackSet)
    if 'name' not in playlist and len(trackSet)>0:
        testType='Challenge'
        qtype='OnlyTracks'
        title=False
        listLen=len(trackSet)
    testPidMeta[playlistId]=(testType,title,qtype,listLen)
    count+=1
print count
    

10000


In [19]:
import os
import subprocess
import pickle
count=0
import pickle
OldTopTracks = pickle.load( open( "../sortedTop1000Tracks.pkl", "rb" ) )
TopTracks=[x.replace('spotify:track:','') for x in OldTopTracks]
submissionFile=open("submission.csv","w+")
submissionFile.write('team_info,main,BachPropagate,surya@ksu.edu\n')
for pid in testPidMeta:
    if pid in rerankedCandidates:
        candTracksList=rerankedCandidates[pid]
    else:
        count+=1
        candTracksList=TopTracks[:500]
    if len(candTracksList)< 500:
        for toptrack in TopTracks:
            if toptrack not in candTracksList:
                candTracksList.append(toptrack)
            if len(candTracksList)>= 500:
                break
    toPrint=['spotify:track:'+x for x in candTracksList[:500]]
    concatTracks= ', '.join(toPrint)
    opLine=pid+', '+concatTracks+'\n'
    submissionFile.write(opLine)
submissionFile.close()  
print count

4
